<a href="https://colab.research.google.com/github/Bayzid03/gemini-chain-of-minds/blob/main/gemini_chain_of_minds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install google-generativeai

In [ ]:
import os
from google.colab import userdata
import google.generativeai as genai

def configure():
    secret_name = "GOOGLE_API_KEY"

    api_key = userdata.get(secret_name)
    if not api_key:
        raise ValueError(f"Secret '{secret_name}' not found. Please check the name in Colab's Secrets.")

    genai.configure(api_key=api_key)
    return genai.GenerativeModel('gemini-1.5-pro-latest')